<a href="https://colab.research.google.com/github/ShadowRaccoon/PrograConcurrente2023C1/blob/main/TP1/TP1_Parte4_Java_M1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Java**

**Enunciado**:
Crear un programa que genere dos hilos, un productor y un consumidor, el productor producirá un valor entero (0 a 99) y se lo comunicará al consumidor a traves de una variable compartida entera.
La idea es que exista una alternancia estricta entre el productor y el consumidor, además ambos hilos deberán comunicar por pantalla cada vez que producen/consumen y el valor correspondiente. Programar dicho comportamiento para producir 10 valores.

In [1]:
%%writefile Buffer.java
import java.util.concurrent.Semaphore;

public class Buffer 
{
    // Valor entero del buffer
    private Integer value;
    
    // Semaforo para sincronizar Productor
    private Semaphore semProducer;
    
    // Semaforo para sicronizar Consumidor
    private Semaphore semConsumer;
    
    public Buffer()
    {
        this.value = null;
        this.semProducer = new Semaphore(1);
        this.semConsumer = new Semaphore(0);
    }
    
    // Se genera valor en el buffer
    public void produce(Integer value) throws InterruptedException
    {
        this.value = value;
    }
    
    // Se lee valor del buffer
    public Integer consume() throws InterruptedException
    {
        Integer value = this.value;
        this.value = null;
        return value;
    }
    
    // P() del semaforo mutex Productor
    public void acquireSemProducer() throws InterruptedException
    {
        semProducer.acquire();
    }
    
    // V() del semaforo mutex Productor
    public void releaseSemProducer()
    {
        semProducer.release();
    }
    
    // P() del semaforo mutex Consumidor
    public void acquireSemConsumer() throws InterruptedException
    {
        semConsumer.acquire();
    }
    
    // V() del semaforo mutex Consumidor
    public void releaseSemConsumer()
    {
        semConsumer.release();
    }
}

Writing Buffer.java


In [2]:
%%writefile Producer.java
public class Producer extends Thread 
{
    // Buffer de comunicación en el hilo
    private Buffer buffer;
    
    private final int NUMBER_OF_REPETITIONS = 10;
    private final int RANGE = 100;

    public Producer(Buffer buffer)
    {
        this.buffer = buffer;
    }
    
    // Método run de la clase Thread. Lógica del hilo
    @Override
    public void run() 
    {
        for (int i = 0; i < this.NUMBER_OF_REPETITIONS; i++) 
        {
            Integer value = (int)(Math.random() * this.RANGE);
            try 
            {
                buffer.acquireSemProducer();
                buffer.produce(value);
                System.out.println(String.format("HILO PRODUCTOR - Deposita valor = %d", value));
                buffer.releaseSemConsumer();
            } 
            catch (InterruptedException ex) 
            {
                System.out.println(String.format("Error al producir %s", ex.getMessage()));
            }
        }
    }
}

Writing Producer.java


In [3]:
%%writefile Consumer.java
public class Consumer extends Thread 
{
    // Buffer de comunicación en el hilo
    private Buffer buffer;
    private final int NUMBER_OF_REPETITIONS = 10;
    
    public Consumer(Buffer buffer)
    {
        this.buffer = buffer;
    }

    // Método run de la clase Thread. Lógica del hilo
    @Override
    public void run() 
    {
        for (int i = 0; i < this.NUMBER_OF_REPETITIONS; i++) 
        {
            try 
            {
                buffer.acquireSemConsumer();
                Integer value = buffer.consume();
                System.out.println(String.format("HILO CONSUMIDOR - Consume valor = %d", value));
                buffer.releaseSemProducer();
            } 
            catch (InterruptedException ex) 
            {
                System.out.println(String.format("Error al consumir %s", ex.getMessage()));
            }
        }
    }
}

Writing Consumer.java


In [4]:
%%writefile Main.java
public class Main 
{

    public static void main(String[] args) throws InterruptedException 
    {
        // Generamos Buffer a utilizar por hilo Productor y Consumidor
        Buffer buffer = new Buffer();
        
        // Generamos hilo Productor e hilo Consumidor
        Producer producer = new Producer(buffer);
        Consumer consumer = new Consumer(buffer);
        
        // Ponemos en ejecución hilo Productor e hilo Consumidor
        producer.start();
        consumer.start();
        
        // Esperamos la finalización de los hilos
        producer.join();
        consumer.join();
    }
}

Writing Main.java


Compilamos los files y que se utilizan clases separadas

In [5]:
!javac Buffer.java
!javac Producer.java
!javac Consumer.java
!javac Main.java

Realizamos la ejecución del Programa:

In [6]:
!java Main

HILO PRODUCTOR - Deposita valor = 28
HILO CONSUMIDOR - Consume valor = 28
HILO PRODUCTOR - Deposita valor = 13
HILO CONSUMIDOR - Consume valor = 13
HILO PRODUCTOR - Deposita valor = 27
HILO CONSUMIDOR - Consume valor = 27
HILO PRODUCTOR - Deposita valor = 56
HILO CONSUMIDOR - Consume valor = 56
HILO PRODUCTOR - Deposita valor = 66
HILO CONSUMIDOR - Consume valor = 66
HILO PRODUCTOR - Deposita valor = 44
HILO CONSUMIDOR - Consume valor = 44
HILO PRODUCTOR - Deposita valor = 80
HILO CONSUMIDOR - Consume valor = 80
HILO PRODUCTOR - Deposita valor = 2
HILO CONSUMIDOR - Consume valor = 2
HILO PRODUCTOR - Deposita valor = 51
HILO CONSUMIDOR - Consume valor = 51
HILO PRODUCTOR - Deposita valor = 25
HILO CONSUMIDOR - Consume valor = 25


Se verifica que se cumple con el enunciado: 
"Crear un programa que genere dos hilos, un productor y un consumidor, el productor producirá un valor entero (0 a 99) y se lo comunicará al consumidor a traves de una variable compartida entera. La idea es que exista una alternancia estricta entre el productor y el consumidor, además ambos hilos deberán comunicar por pantalla cada vez que producen/consumen y el valor correspondiente. Programar dicho comportamiento para producir 10 valores."